In [ ]:
import tensorflow.lite as tflite

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import tensorflow_datasets as tfds
from time import perf_counter

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
from model.ModelBuilder import ModelBuilder
import json
import tensorflow as tf

######################################### MODEL
modelName = "MobileNetV3_PFH_SSD"

model_dir = "checkpoints/"
modelPart = modelName.split("_")

with open(os.path.join("model", "0_Config", modelName+".json"), "r") as config_file:
    config = json.load(config_file)

config['modelName'] = modelName
model = ModelBuilder(config = config)


In [ ]:
class Detector():
    def __init__(self):
        self._max_results = 30
        
        self._interpreter = tflite.Interpreter(model_path='converted_model.tflite', num_threads=4)
        
        input_detail = self._interpreter.get_input_details()[0]
        self._interpreter.allocate_tensors()
        self._model_input_size = (input_detail['shape'][1], input_detail['shape'][2])
    
        self._is_quantized_input = input_detail['dtype'] == np.uint8

        self.scale, self.zero_point = self._interpreter.get_input_details()[0]['quantization']


        sorted_output_indices = sorted([output['index'] for output in self._interpreter.get_output_details()])

        self._output_indices = {
            'BBOX': sorted_output_indices[0],
            'CLASS': sorted_output_indices[1],
            'SCORE': sorted_output_indices[2],
            'VALIDNUM': sorted_output_indices[3], 
        }

    def detect(self, input_image):
        input_tensor = self._preprocess(input_image)
        self._set_input_tensor(input_tensor)
        self._interpreter.invoke()
        return self._postprocess()

    def _preprocess(self, input_image):
        input_tensor = cv2.resize(input_image, self._model_input_size)
        
        if self._is_quantized_input:
            #input_tensor = input_tensor / self.scale + self.zero_point
            pass
        else:
            input_tensor = input_tensor/127.5-1

        return np.expand_dims(input_tensor, axis=0)

    def _set_input_tensor(self, image):
        tensor_index = self._interpreter.get_input_details()[0]['index']
        input_tensor = self._interpreter.tensor(tensor_index)()[0]
        input_tensor[:, :] = image

    def _get_output_tensor(self, name):
        output_index = self._output_indices[name]
        return np.squeeze(self._interpreter.get_tensor(output_index))

    def _postprocess(self):
        bboxes = self._get_output_tensor('BBOX')[:self._max_results, :] #max
        cls = self._get_output_tensor('CLASS')[:self._max_results] #c
        scores = self._get_output_tensor('SCORE')[:self._max_results] #b
        return bboxes, cls, scores

In [ ]:
[test_dataset], dataset_info = tfds.load(name="coco/2017", split=["validation"], with_info=True)

In [ ]:
labelMap_Func = dataset_info.features["objects"]["label"].int2str
colors = np.random.rand(200, 3)*255
detector = Detector()
score_threshold = 0.5
numPic = 10

TimeBench = {'FPS_Inf':0.0}

for sample in test_dataset.take(numPic):
    plt.figure(figsize=(12,12))
    orignal_image = sample['image'].numpy()
    input_img = orignal_image
    
    ground_truth = sample['objects']['bbox']
    
    detection_start = perf_counter()
    bboxes, classes, scores = detector.detect(input_img)
    detection_end = perf_counter()
    
    TimeBench['FPS_Inf'] += 1/(detection_end - detection_start)
    for bbox, cls, score in zip(bboxes, classes, scores):
        y1, x1, y2, x2 = bbox
        
        if score < score_threshold:
            break
        x1 = int(x1*orignal_image.shape[1])
        x2 = int(x2*orignal_image.shape[1])
        y1 = int(y1*orignal_image.shape[0])
        y2 = int(y2*orignal_image.shape[0])
        _text = '{}_{:.2f}'.format(labelMap_Func(int(cls)), score)
        #_text = '{:.2f}'.format(score)
        cv2.rectangle(orignal_image, (x1, y1), (x2, y2), colors[int(cls)], 1)
        cv2.putText(orignal_image, _text, (x1,y1+10), cv2.FONT_HERSHEY_COMPLEX, 0.4, colors[int(cls)], thickness=1, lineType=cv2.LINE_AA)

    plt.imshow(orignal_image)

TimeBench['FPS_Inf'] /= numPic

In [ ]:
TimeBench